<a href="https://colab.research.google.com/github/Boris-Kuliev/My_First_Test_Repo/blob/main/code/%D0%9B%D0%A02_%D0%B2%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5_%D1%80%D1%8F%D0%B4%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd     # для чтения и записи табличных файлов
import numpy as np      # для математической обработки данных
# для создания интерактивных графиков
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

# Получение и чтение файла из облака
url = "https://drive.google.com/uc?export=download&id=1y94Eid5lB29n8e9ojVRG3zbTiHO_xm4i"

# чтение данных, разделитель столбцов можно не указывать, Pandas при чтении XLS файла сам его определит; 
# имена у столбцов в исходном файле есть, в первой строке, поэтому нужно указать header = 0.
input_df = pd.read_excel(url, header = 0) 

# вывод заголовка таблицы и первых пяти строк (сэмплов)
print(input_df.head(5))
print("...")
print(input_df.tail(5))

# Берем только временной ряд (почасовые значения) скорости ветра в м/с
# iloc - выделяет из таблицы указанный фрагмент, в данном случае все строки и только первый(0) столбец
wind_speed = input_df.iloc[: , 0].values

output_notebook()

# Создаем график среднесуточной выработки ВЭС
p = figure(plot_width = 1000, plot_height = 600)
p.line(np.arange(len(wind_speed)), wind_speed, line_width = 2, color = 'green')
p.xaxis.axis_label = 'Время, часы'
p.yaxis.axis_label = 'Скорость ветра, м/с'
show(p)

# Зададим зависимость среднечасовой мощности, вырабатываемой ВЭУ, от средней скорости ветра
def wind_speed_to_power(speed):
  min_speed = 5
  max_speed = 20
  critical_speed = 25
  
  # выработка равна нулю, если скорость ветра выше критической или ниже минимальной
  # поэтому обнуляем такую скорость
  speed_tmp = np.where(np.logical_or(speed > critical_speed, speed < min_speed), 0, speed)

  # если скорость выше максимальной, то выработка не будет расти, добавляем ограничение сверху 
  speed_tmp = np.clip(speed_tmp, a_min = None, a_max = max_speed)

  # на рабочем участке от min_speed до max_speed мощность пропорциональна кубу скорости
  power = 0.0017 * (speed_tmp) ** 3
  return power

# отдельно создается график зависимости выработки от скорости ветра
x = np.arange(0, 30, 0.1)
y = [wind_speed_to_power(i) for i in x]
p = figure(plot_width = 600, plot_height = 400)
p.line(x, y, line_width = 2, color = 'blue')
p.xaxis.axis_label = 'Скорость ветра, м/с'
p.yaxis.axis_label = 'Мощность ВЭУ, МВт'
show(p)

# Рассчитываем почасовые значения мощности
power = [wind_speed_to_power(v) for v in wind_speed]

# Создаем график зависимости вырабатываемой мощности ВЭС от скорости ветра и временной график генерации
p = figure(plot_width = 1000, plot_height = 600)
times = np.arange(len(wind_speed))
p.line(times, power, line_width = 2, color = 'blue', legend_label= 'Мощность, МВт')
p.line(times, wind_speed * 0.1, line_width = 2, color = 'orange', legend_label= 'Скорость, 0.1 м/с')
p.xaxis.axis_label = 'Время, часы'
p.legend.location = "top_right"
show(p)

# Делим почасовые значения мощности на подмассивы по суткам
powerByDays = [power[i:i + 24] for i in range(0, len(power), 24)]
# Рассчитываем среднюю мощность за каждые сутки
avgDailyPower = [np.average(p) for p in powerByDays]

# Создаем график среднесуточной выработки ВЭС
p = figure(plot_width = 1000, plot_height = 600)
p.line(np.arange(len(avgDailyPower)), avgDailyPower, line_width = 2, color = 'blue')
p.xaxis.axis_label = 'Время, сутки'
p.yaxis.axis_label = 'Среднесуточная мощность ВЭУ, МВт'
show(p)


# полученные данные можно сохранить в файл в Excel формат
data_frame = pd.DataFrame(power)
data_frame.to_excel('power_by_hours.xlsx', index = False, header = ['Мощность, МВт'])
data_frame = pd.DataFrame(avgDailyPower)
data_frame.to_excel('average_power_by_days.xlsx', index = False, header = ['Среднесуточная мощность ВЭУ, МВт'])


   Wind, m/s
0      15.18
1      15.84
2      16.60
3      14.98
4      13.28
...
     Wind, m/s
715       4.94
716       2.84
717       2.56
718       2.18
719       1.80
